TASK B

In [1]:
import pandas as pd
import numpy as np
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roshnidas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [169]:
posts = pd.read_csv('gofundme.csv')

In [170]:
posts['post_created_on']=pd.to_datetime(posts['post_created_on'])

In [ ]:
import re
import matplotlib.pyplot as plt
from google.oauth2 import service_account
from google.cloud import vision
import csv
import urllib.request
import time

DAYS_GRAPH_MAX = 730
MAX_POSTS = 5
DONATIONS_URL = 'https://www.gofundme.com/mvc.php?route=donate/pagingDonationsFoundation&url={}&idx={}&type=recent'
SCALE_DIFFERENTIATING_AMOUNT = 25000

class Donation:
    def __init__(self, time_raised, amount):
        self.time_raised = time_raised
        self.amount = amount

class Fundraise:
    def __init__(self, id, start_url, url, post_name, story, amount_donated, amount_goal, amount_donators, likes, shares, location, created_date, post_image):
        self.id = id
        self.start_url = start_url
        self.url = url
        self.post_name = post_name
        self.story = story
        self.amount_donated = amount_donated
        self.amount_goal = amount_goal
        self.amount_donators = amount_donators
        self.likes = likes
        self.shares = shares
        self.location = location
        self.created_date = created_date
        self.post_image = post_image
        self.image_labels = []

donations = []


def import_scraped_posts(file_path):
    post_counter = 0
    fundraises = []
    with open(file_path,'r', encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        next(csv_reader)
        for row in csv_reader:
            if post_counter > MAX_POSTS:
                break
            fundraise = Fundraise(row[0],row[1],row[3],row[4],row[5],re.sub('[^0-9]','', row[6]),re.sub('[^0-9]','', row[7]),row[8],row[9],row[10],row[11],row[12],row[13])
            fundraises.append(fundraise)
            post_counter += 1
    return fundraises

def normalize_time_raised(time_raised):
    time_raised_str = time_raised
    if 'hours' in time_raised_str:
        time_raised = 0
    p = re.compile(r"\d+")
    match = p.match(time_raised)
    if match:
        time_raised = int(match[0])
    if 'month' in time_raised_str:
        time_raised = time_raised * 30
    return time_raised

def analyze_donation_times(donations):
    days_graph = [0] * DAYS_GRAPH_MAX
    for i in range(len(donations)):
        time_raised = donations[i].time_raised
        time_raised = normalize_time_raised(time_raised)
        donations[i].time_raised = time_raised

    for donation in donations:
        if donation.time_raised < DAYS_GRAPH_MAX:
            days_graph[donation.time_raised] += 1
    return days_graph

def plot_donations_graph(days_graph):
    plt.plot(days_graph)
    plt.ylabel('donators')
    plt.xlabel('days')
    plt.show()

def get_posts_attrs(fundraises):
    credentials = service_account.Credentials.from_service_account_file('UGCA-f6b4b2de52f8.json')
    client = vision.ImageAnnotatorClient(credentials=credentials)
    image = vision.types.Image()
    for fundraise in fundraises:
        if fundraise.post_image == '':
            continue
        image.source.image_uri = fundraise.post_image
        response = client.label_detection(image=image)
        fundraise.image_labels = [label.description for label in response.label_annotations]

def extract_donations(html):
    donation_times = []

    donation_time_strs = re.findall(r"([0-9]+? [a-z]{3,8}) ago", html)
    donation_times = [normalize_time_raised(donation_time_str) for donation_time_str in donation_time_strs]
    return donation_times

def get_fundraise_donations(project_url_name):
    donation_times = []
    index = 0

    response = urllib.request.urlopen(DONATIONS_URL.format(project_url_name, index))
    html = str(response.read())
    while len(html)>800:
        start_time = time.time()
        response = urllib.request.urlopen(DONATIONS_URL.format(project_url_name,index))
        print("--- %s seconds ---" % (time.time() - start_time))
        html = str(response.read())
        donation_times.extend(extract_donations(html))
        index += 10
    return donation_times

def get_fundraise_donations_to_file(project_url_name):
    donation_times = []
    index = 0

    response = urllib.request.urlopen(DONATIONS_URL.format(project_url_name, index))
    html = str(response.read())
    while len(html)>800:
        start_time = time.time()
        response = urllib.request.urlopen(DONATIONS_URL.format(project_url_name,index))
        print("--- %s seconds ---" % (time.time() - start_time))
        html = str(response.read())
        donation_times.extend(extract_donations(html))
        index += 10
    with open(project_url_name, 'w') as out_file:
        for donation_time in donation_times:
            out_file.write("%s\n" % str(donation_time))
    print(donation_times)
    return donation_times

def get_donation_times(fundraises):
    donation_times = []
    i=0
    fundraises.sort(key= lambda fundraise: fundraise.amount_donated/fundraise.amount_goal, reverse=True)
    fundraises = fundraises[0:100]
    fundraises.sort(key= lambda fundraise: fundraise.amount_donated, reverse=True)

    while i in range (1) and i in range(len(fundraises)):
        p = re.compile(r".*?\/([^\/]*)$")
        match = p.match(fundraises[i].url)
        if match:
            project_url_name = match[1]
        else:
            i += 1
            continue
        donation_times.extend(get_fundraise_donations(fundraises[i],project_url_name))
        i += 1

    return donation_times


fundraises = import_scraped_posts('gofundme.csv')

#small_scale_fundraising = filter(lambda fundraise: fundraise.amount_goal < SCALE_DIFFERENTIATING_AMOUNT, fundraises)
#large_scale_fundraising = filter(lambda fundraise: fundraise.amount_goal >= SCALE_DIFFERENTIATING_AMOUNT, fundraises)

#donation_times = get_donation_times(fundraises)
#print(donation_times)
#get_posts_attrs(fundraises)

#print(fundraises[1].image_labels)

#days_graph = analyze_donation_times(donations)

get_fundraise_donations_to_file('lougheed')

In [171]:
posts.shape

(765, 15)

In [172]:
for index,row in posts.iterrows():
    if posts['amount_goal'][index]=='goal':
        posts['amount_goal'][index]=posts['amount_donated'][index]
        posts['amount_donated'][index]=0
        posts['donatedby_pplcount'][index]=0

In [173]:
posts['likes']=posts['likes'].fillna(0)
posts['shares']=posts['shares'].fillna(0)

In [174]:
posts.amount_donated = posts.amount_donated.astype(int)

In [175]:
posts

,Unnamed: 0,web-scraper-order,web-scraper-start-url,url,url-href,post_name,story,amount_donated,amount_goal,donatedby_pplcount,likes,shares,location,post_created_on,image-src
0,0,1543888757-33644,https://www.gofundme.com/mvc.php?route=homepag...,IBEW Members Who Lost Homes to Fire\n ...,https://www.gofundme.com/IBEW1245Fire,IBEW Members Who Lost Homes to Fire,Several IBEW Local 1245 members and retirees h...,7390,50000.00,49,50,75,"VACAVILLE, CA",2015-09-16,https://2dbdd5116ffa30a49aa8-c03f075f8191fb4e6...
1,1,1543888416-33490,https://www.gofundme.com/mvc.php?route=homepag...,Cardinal Newman High School Prom\n ...,https://www.gofundme.com/cardinal-newman-high-...,Cardinal Newman High School Prom,"On October 9th, 2017 when the Tubbs fire burne...",0,28630.00,0,0,7,"WALNUT CREEK, CA",2018-02-10,https://d2g8igdw686xgo.cloudfront.net/27651948...
2,2,1543889806-34056,https://www.gofundme.com/mvc.php?route=homepag...,Home lost in California wildfire\n ...,https://www.gofundme.com/home-lost-in-californ...,Home lost in California wildfire,David Porter and his family lost their home an...,9350,25000.00,62,61,172,"PARADISE, CA",2018-11-10,https://d2g8igdw686xgo.cloudfront.net/34405786...
3,3,1543889785-34047,https://www.gofundme.com/mvc.php?route=homepag...,California wildfire relief\n ...,https://www.gofundme.com/hafsz-california-wild...,California wildfire relief,"At 12:30 am, Sunday, October 8, I was awakened...",4615,5000.00,33,32,61,"GLEN ELLEN, CA",2017-10-18,https://d2g8igdw686xgo.cloudfront.net/24673878...
4,4,1543889728-34022,https://www.gofundme.com/mvc.php?route=homepag...,Help California Wildfire Victims\n ...,https://www.gofundme.com/3cwfv8-help-californi...,Help California Wildfire Victims,"Hello y'all, \n\nAs many of you know, our frie...",110,500.00,4,4,37,"LAKEWOOD, OH",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34518224...
5,5,1543888215-33400,https://www.gofundme.com/mvc.php?route=homepag...,Support Wildfires Of California\n ...,https://www.gofundme.com/support-wildfires-of-...,Support Wildfires Of California,A couple days ago a huge wildfire started in c...,0,300.00,0,0,0,"COLLEYVILLE, TX",2017-12-14,https://d2g8igdw686xgo.cloudfront.net/26224664...
6,6,1543889649-33989,https://www.gofundme.com/mvc.php?route=homepag...,California Wildfire Donation Drive\n ...,https://www.gofundme.com/california-wildfire-d...,California Wildfire Donation Drive,A colleague of mine had her family's home comp...,50,1500.00,1,1,4,"SAN JOSE, CA",2018-11-14,NaN
7,7,1543888990-33721,https://www.gofundme.com/mvc.php?route=homepag...,Blank Family Santa Rosa Fire Fund\n ...,https://www.gofundme.com/blank-santa-rosa-fire...,Blank Family Santa Rosa Fire Fund,"Our parents, Gary Blank and Carole Cooper, jus...",12780,30000.00,124,127,527,"SANTA ROSA, CA",2017-10-11,https://d2g8igdw686xgo.cloudfront.net/24575816...
8,8,1543889006-33728,https://www.gofundme.com/mvc.php?route=homepag...,A new home for Lindsay and Riley\n ...,https://www.gofundme.com/lindsayandriley,A new home for Lindsay and Riley,We are reaching out to help our dear friend Li...,29332,20000.00,130,138,661,"SANTA ROSA, CA",2017-10-19,https://d2g8igdw686xgo.cloudfront.net/24765678...
9,9,1543888829-33657,https://www.gofundme.com/mvc.php?route=homepag...,North Valley Animal Disaster Group\n ...,https://www.gofundme.com/north-valley-animal-d...,North Valley Animal Disaster Group,Vets and nurses from the North Valley Animal D...,500,500.00,19,18,33,"DUNEDIN, FL",2018-11-15,https://d2g8igdw686xgo.cloudfront.net/34544892...


In [176]:
#posts['amount_donated'] = [int(amount[1:]) for amount in posts['amount_donated']] #.replace(',',''))
goal = []

for index,row in posts.iterrows():
    if posts['amount_goal'][index] == 'raised':
        goal.append(posts['amount_donated'][index])
    else:
        #a = gathered[4:-5].replace(',','')
        if 'M' in posts['amount_goal'][index]:
            b = float(posts['amount_goal'][index][:-1].replace('M',''))*100000
            goal.append(b)
        else:
            #print(gathered)
            goal.append(float(posts['amount_goal'][index]))


posts['amount_goal'] = goal
posts['percentage'] = (posts['amount_donated'] / posts['amount_goal'])*100

In [177]:
posts

,Unnamed: 0,web-scraper-order,web-scraper-start-url,url,url-href,post_name,story,amount_donated,amount_goal,donatedby_pplcount,likes,shares,location,post_created_on,image-src,percentage
0,0,1543888757-33644,https://www.gofundme.com/mvc.php?route=homepag...,IBEW Members Who Lost Homes to Fire\n ...,https://www.gofundme.com/IBEW1245Fire,IBEW Members Who Lost Homes to Fire,Several IBEW Local 1245 members and retirees h...,7390,50000.0,49,50,75,"VACAVILLE, CA",2015-09-16,https://2dbdd5116ffa30a49aa8-c03f075f8191fb4e6...,14.780000
1,1,1543888416-33490,https://www.gofundme.com/mvc.php?route=homepag...,Cardinal Newman High School Prom\n ...,https://www.gofundme.com/cardinal-newman-high-...,Cardinal Newman High School Prom,"On October 9th, 2017 when the Tubbs fire burne...",0,28630.0,0,0,7,"WALNUT CREEK, CA",2018-02-10,https://d2g8igdw686xgo.cloudfront.net/27651948...,0.000000
2,2,1543889806-34056,https://www.gofundme.com/mvc.php?route=homepag...,Home lost in California wildfire\n ...,https://www.gofundme.com/home-lost-in-californ...,Home lost in California wildfire,David Porter and his family lost their home an...,9350,25000.0,62,61,172,"PARADISE, CA",2018-11-10,https://d2g8igdw686xgo.cloudfront.net/34405786...,37.400000
3,3,1543889785-34047,https://www.gofundme.com/mvc.php?route=homepag...,California wildfire relief\n ...,https://www.gofundme.com/hafsz-california-wild...,California wildfire relief,"At 12:30 am, Sunday, October 8, I was awakened...",4615,5000.0,33,32,61,"GLEN ELLEN, CA",2017-10-18,https://d2g8igdw686xgo.cloudfront.net/24673878...,92.300000
4,4,1543889728-34022,https://www.gofundme.com/mvc.php?route=homepag...,Help California Wildfire Victims\n ...,https://www.gofundme.com/3cwfv8-help-californi...,Help California Wildfire Victims,"Hello y'all, \n\nAs many of you know, our frie...",110,500.0,4,4,37,"LAKEWOOD, OH",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34518224...,22.000000
5,5,1543888215-33400,https://www.gofundme.com/mvc.php?route=homepag...,Support Wildfires Of California\n ...,https://www.gofundme.com/support-wildfires-of-...,Support Wildfires Of California,A couple days ago a huge wildfire started in c...,0,300.0,0,0,0,"COLLEYVILLE, TX",2017-12-14,https://d2g8igdw686xgo.cloudfront.net/26224664...,0.000000
6,6,1543889649-33989,https://www.gofundme.com/mvc.php?route=homepag...,California Wildfire Donation Drive\n ...,https://www.gofundme.com/california-wildfire-d...,California Wildfire Donation Drive,A colleague of mine had her family's home comp...,50,1500.0,1,1,4,"SAN JOSE, CA",2018-11-14,NaN,3.333333
7,7,1543888990-33721,https://www.gofundme.com/mvc.php?route=homepag...,Blank Family Santa Rosa Fire Fund\n ...,https://www.gofundme.com/blank-santa-rosa-fire...,Blank Family Santa Rosa Fire Fund,"Our parents, Gary Blank and Carole Cooper, jus...",12780,30000.0,124,127,527,"SANTA ROSA, CA",2017-10-11,https://d2g8igdw686xgo.cloudfront.net/24575816...,42.600000
8,8,1543889006-33728,https://www.gofundme.com/mvc.php?route=homepag...,A new home for Lindsay and Riley\n ...,https://www.gofundme.com/lindsayandriley,A new home for Lindsay and Riley,We are reaching out to help our dear friend Li...,29332,20000.0,130,138,661,"SANTA ROSA, CA",2017-10-19,https://d2g8igdw686xgo.cloudfront.net/24765678...,146.660000
9,9,1543888829-33657,https://www.gofundme.com/mvc.php?route=homepag...,North Valley Animal Disaster Group\n ...,https://www.gofundme.com/north-valley-animal-d...,North Valley Animal Disaster Group,Vets and nurses from the North Valley Animal D...,500,500.0,19,18,33,"DUNEDIN, FL",2018-11-15,https://d2g8igdw686xgo.cloudfront.net/34544892...,100.000000


In [178]:
import datetime

today=datetime.datetime.today()
posts['life_of_post']=0
for index, row in posts.iterrows():
    #print(index)
    posts['life_of_post'][index]=(today-posts['post_created_on'][index]).days
   # print(posts['life_of_post'][index])
    
#=(today-posts['post__on']).days
# posts['life_of_post']
# posts['post__on']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [182]:
#gapminder_not_2002 = gapminder[gapminder.year != 2002]
#filtered_posts = posts[posts.percentage>=5]

filtered_posts=posts[(posts.life_of_post>16)|(posts.percentage>5)]  
#filtered_posts=filtered_posts[filtered_posts.percentage>5]

In [183]:
filtered_posts.shape

(670, 17)

In [43]:
#posts.to_csv('gofundme.csv', sep=',')

Dividing into Small and Large Scale projects:

In [189]:
THRESHOLD=10000
small_scale_funds=filtered_posts[filtered_posts.amount_goal<=THRESHOLD]
large_scale_funds=filtered_posts[filtered_posts.amount_goal>THRESHOLD]

**SMALL SCALE:**

In [190]:
small_scale_funds.shape

(483, 17)

In [191]:
large_scale_funds.shape

(187, 17)

In [192]:
def set_missing(df):
  
    known = df[df.story.notnull()]
    unknown = df[df.story.isnull()]
    var = known.columns.tolist()
    X = known.loc[:, var ].drop('story',axis = 1)
    y = known['story']
    # replace predicted values
    predicted = ' '
    df.loc[(df.story.isnull()), 'story'] = predicted
    return df.info()

set_missing(small_scale_funds)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483 entries, 3 to 764
Data columns (total 17 columns):
Unnamed: 0               483 non-null int64
web-scraper-order        483 non-null object
web-scraper-start-url    483 non-null object
url                      483 non-null object
url-href                 483 non-null object
post_name                483 non-null object
story                    483 non-null object
amount_donated           483 non-null int64
amount_goal              483 non-null float64
donatedby_pplcount       473 non-null object
likes                    483 non-null object
shares                   483 non-null object
location                 453 non-null object
post_created_on          453 non-null datetime64[ns]
image-src                460 non-null object
percentage               483 non-null float64
life_of_post             453 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(11)
memory usage: 67.9+ KB


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [193]:
def count_words(dataframe,column):
    captions = []
    dataframe = dataframe.reset_index(drop=True)
    for i in range(dataframe.shape[0]):
        captions.append(pd.DataFrame(dataframe.iloc[:][column]).loc[i][0].lower())
    wordcount = Counter(pos_tag(word_tokenize(''.join(captions))))
    word_list = sorted(list(wordcount.items()), key = lambda w: -w[1])
    stoplist = nltk.corpus.stopwords.words('english')
    word_list = [word_list[i] for i in range(len(word_list)) 
                 if len(word_list[i][0][0]) > 2 and word_list[i][0][0] not in stoplist 
                 and word_list[i][0][1] in ['NN', 'NNP', 'NNS', 'JJ']]
    return word_list

count_words(small_scale_funds.fillna('0'), 'post_name')

[(('fire', 'NN'), 102),
 (('wildfire', 'NN'), 79),
 (('california', 'NN'), 56),
 (('relief', 'NN'), 38),
 (('family', 'NN'), 24),
 (('wild', 'JJ'), 16),
 (('paradise', 'NN'), 12),
 (('camp', 'NN'), 11),
 (('victims', 'NNS'), 11),
 (('wildfirescalifornia', 'NN'), 10),
 (('reliefcalifornia', 'NN'), 10),
 (('home', 'NN'), 10),
 (('help', 'NN'), 9),
 (('house', 'NN'), 8),
 (('animal', 'JJ'), 7),
 (('fund', 'NN'), 7),
 (('wildfires', 'NNS'), 6),
 (('firecalifornia', 'NN'), 6),
 (('fundcalifornia', 'NN'), 6),
 (('wildfirecalifornia', 'NN'), 6),
 (('everything', 'NN'), 6),
 (('recovery', 'NN'), 6),
 (('reliefhelp', 'NN'), 5),
 (('victimscalifornia', 'NN'), 5),
 (('camp', 'JJ'), 4),
 (('familycalifornia', 'NN'), 4),
 (('fires', 'NNS'), 4),
 (('support', 'NN'), 4),
 (('cali', 'NN'), 4),
 (('emergency', 'NN'), 4),
 (('california', 'JJ'), 4),
 (('valley', 'NN'), 3),
 (('firehelp', 'NN'), 3),
 (('fundwildfire', 'NN'), 3),
 (('students', 'NNS'), 3),
 (('animal', 'NN'), 3),
 (('start', 'NN'), 3),
 (

In [195]:
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.tokenize import word_tokenize
# from math import log
# import numpy as np
# import csv
# import re
# from collections import namedtuple


from nltk.sentiment.vader import SentimentIntensityAnalyzer
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


def get_story_sentiment_score(stories):
    sentiments = []
    analyser = SentimentIntensityAnalyzer()
    for story in stories:
        story_sentiment = analyser.polarity_scores(story)['compound']
        sentiments.append(story_sentiment)
    return sentiments

small_scale_funds['sentiment'] = get_story_sentiment_score(small_scale_funds['story'].fillna('0'))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [209]:
#small_scale_funds['percent_life_ratio']=small_scale_funds['percentage']/small_scale_funds['life_of_post']
small_scale_funds['success']=0
for index, row in small_scale_funds.iterrows():
    if small_scale_funds['percentage'][index]>70 :
        if (small_scale_funds['life_of_post'][index]<60) | (small_scale_funds['percentage'][index]>90):
            small_scale_funds['success'][index]=1
        else:
            small_scale_funds['success'][index]=0

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [210]:
small_scale_funds

,Unnamed: 0,web-scraper-order,web-scraper-start-url,url,url-href,post_name,story,amount_donated,amount_goal,donatedby_pplcount,likes,shares,location,post_created_on,image-src,percentage,life_of_post,sentiment,percent_life_ratio,success
3,3,1543889785-34047,https://www.gofundme.com/mvc.php?route=homepag...,California wildfire relief\n ...,https://www.gofundme.com/hafsz-california-wild...,California wildfire relief,"At 12:30 am, Sunday, October 8, I was awakened...",4615,5000.0,33,32,61,"GLEN ELLEN, CA",2017-10-18,https://d2g8igdw686xgo.cloudfront.net/24673878...,92.300000,411.0,0.9522,0.224574,1
4,4,1543889728-34022,https://www.gofundme.com/mvc.php?route=homepag...,Help California Wildfire Victims\n ...,https://www.gofundme.com/3cwfv8-help-californi...,Help California Wildfire Victims,"Hello y'all, \n\nAs many of you know, our frie...",110,500.0,4,4,37,"LAKEWOOD, OH",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34518224...,22.000000,19.0,0.7717,1.157895,0
5,5,1543888215-33400,https://www.gofundme.com/mvc.php?route=homepag...,Support Wildfires Of California\n ...,https://www.gofundme.com/support-wildfires-of-...,Support Wildfires Of California,A couple days ago a huge wildfire started in c...,0,300.0,0,0,0,"COLLEYVILLE, TX",2017-12-14,https://d2g8igdw686xgo.cloudfront.net/26224664...,0.000000,354.0,0.3382,0.000000,0
6,6,1543889649-33989,https://www.gofundme.com/mvc.php?route=homepag...,California Wildfire Donation Drive\n ...,https://www.gofundme.com/california-wildfire-d...,California Wildfire Donation Drive,A colleague of mine had her family's home comp...,50,1500.0,1,1,4,"SAN JOSE, CA",2018-11-14,NaN,3.333333,19.0,-0.5371,0.175439,0
9,9,1543888829-33657,https://www.gofundme.com/mvc.php?route=homepag...,North Valley Animal Disaster Group\n ...,https://www.gofundme.com/north-valley-animal-d...,North Valley Animal Disaster Group,Vets and nurses from the North Valley Animal D...,500,500.0,19,18,33,"DUNEDIN, FL",2018-11-15,https://d2g8igdw686xgo.cloudfront.net/34544892...,100.000000,18.0,0.9606,5.555556,1
10,10,1543888880-33677,https://www.gofundme.com/mvc.php?route=homepag...,Help Gail and Gene Rebuild!\n ...,https://www.gofundme.com/3t6wcnk,Help Gail and Gene Rebuild!,The Northern California wildfires have devasta...,5405,10000.0,56,56,224,"PARK CITY, UT",2017-10-09,https://d2g8igdw686xgo.cloudfront.net/20533300...,54.050000,420.0,0.9819,0.128690,0
11,11,1543889268-33835,https://www.gofundme.com/mvc.php?route=homepag...,Southern California Wildfire Relief\n ...,https://www.gofundme.com/southern-california-w...,Southern California Wildfire Relief,"Throughout the past week, raging wildfires hav...",0,1000.0,0,0,0,"SAN RAFAEL, CA",2017-12-06,https://d2g8igdw686xgo.cloudfront.net/25968498...,0.000000,362.0,-0.8241,0.000000,0
12,12,1543889050-33747,https://www.gofundme.com/mvc.php?route=homepag...,Fire in Paradise\n Fire...,https://www.gofundme.com/fire-in-paradise,Fire in Paradise,My son Tony(Robert) and his partner Aine have ...,4881,5000.0,77,75,262,"SEATTLE, WA",2018-11-08,https://d2g8igdw686xgo.cloudfront.net/34351750...,97.620000,25.0,0.6032,3.904800,1
15,15,1543888691-33617,https://www.gofundme.com/mvc.php?route=homepag...,Shifflet employees\n Sh...,https://www.gofundme.com/shifflet-employees,Shifflet employees,The California wildfires have claimed the home...,4251,1500.0,22,23,52,"HELENA, MT",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34516058...,283.400000,19.0,-0.6369,14.915789,1
16,16,1543889096-33766,https://www.gofundme.com/mvc.php?route=homepag...,Send Wildfire Survivors for Climate Action to ...,https://www.gofundme.com/send-wildfire-survivo...,Send Wildfire Survivors for Climate Action to DC,In a week we've assembled an INCREDIBLE grassr...,1196,4200.0,22,22,40,"LOS ANGELES, CA",2018-11-20,https://d2g8igdw686xgo.cloudfront.net/34684456...,28.476190,13.0,0.9865,2.190476,0


**LARGE SCALE:**

In [211]:
set_missing(large_scale_funds)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 762
Data columns (total 17 columns):
Unnamed: 0               187 non-null int64
web-scraper-order        187 non-null object
web-scraper-start-url    187 non-null object
url                      187 non-null object
url-href                 187 non-null object
post_name                187 non-null object
story                    187 non-null object
amount_donated           187 non-null int64
amount_goal              187 non-null float64
donatedby_pplcount       187 non-null object
likes                    187 non-null object
shares                   187 non-null object
location                 184 non-null object
post_created_on          184 non-null datetime64[ns]
image-src                178 non-null object
percentage               187 non-null float64
life_of_post             184 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(11)
memory usage: 26.3+ KB


In [212]:
count_words(large_scale_funds.fillna('0'), 'post_name')

[(('fire', 'NN'), 40),
 (('california', 'NN'), 27),
 (('wildfire', 'NN'), 22),
 (('family', 'NN'), 15),
 (('relief', 'NN'), 14),
 (('wild', 'JJ'), 10),
 (('home', 'NN'), 9),
 (('camp', 'NN'), 8),
 (('help', 'NN'), 5),
 (('victims', 'NNS'), 5),
 (('recovery', 'NN'), 5),
 (('wildfirescalifornia', 'NN'), 4),
 (('fundhelp', 'NN'), 3),
 (('paradise', 'NN'), 3),
 (('homes', 'NNS'), 2),
 (('high', 'JJ'), 2),
 (('school', 'NN'), 2),
 (('firecalifornia', 'NN'), 2),
 (('medical', 'JJ'), 2),
 (('reliefcalifornia', 'NN'), 2),
 (('wildfires', 'NNS'), 2),
 (('hope', 'NN'), 2),
 (('fireparadise', 'NN'), 2),
 (('fundcalifornia', 'NN'), 2),
 (('wildfirecalifornia', 'NN'), 2),
 (('families', 'NNS'), 2),
 (('camp', 'JJ'), 2),
 (('fund', 'NN'), 2),
 (('house', 'NN'), 2),
 (('rescue', 'NN'), 2),
 (('bear', 'JJ'), 2),
 (('ibew', 'JJ'), 1),
 (('members', 'NNS'), 1),
 (('firecardinal', 'JJ'), 1),
 (('newman', 'JJ'), 1),
 (('promhome', 'NN'), 1),
 (('wildfireblank', 'NN'), 1),
 (('rosa', 'JJ'), 1),
 (('new', '

In [213]:
large_scale_funds['sentiment'] = get_story_sentiment_score(large_scale_funds['story'].fillna('0'))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [214]:
#small_scale_funds['percent_life_ratio']=small_scale_funds['percentage']/small_scale_funds['life_of_post']
large_scale_funds['success']=0
for index, row in large_scale_funds.iterrows():
    if large_scale_funds['percentage'][index]>70 :
        if (large_scale_funds['life_of_post'][index]<60) | (large_scale_funds['percentage'][index]>90):
            large_scale_funds['success'][index]=1
        else:
            large_scale_funds['success'][index]=0

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [215]:
large_scale_funds

,Unnamed: 0,web-scraper-order,web-scraper-start-url,url,url-href,post_name,story,amount_donated,amount_goal,donatedby_pplcount,likes,shares,location,post_created_on,image-src,percentage,life_of_post,sentiment,success
0,0,1543888757-33644,https://www.gofundme.com/mvc.php?route=homepag...,IBEW Members Who Lost Homes to Fire\n ...,https://www.gofundme.com/IBEW1245Fire,IBEW Members Who Lost Homes to Fire,Several IBEW Local 1245 members and retirees h...,7390,50000.0,49,50,75,"VACAVILLE, CA",2015-09-16,https://2dbdd5116ffa30a49aa8-c03f075f8191fb4e6...,14.780000,1174.0,-0.2675,0
1,1,1543888416-33490,https://www.gofundme.com/mvc.php?route=homepag...,Cardinal Newman High School Prom\n ...,https://www.gofundme.com/cardinal-newman-high-...,Cardinal Newman High School Prom,"On October 9th, 2017 when the Tubbs fire burne...",0,28630.0,0,0,7,"WALNUT CREEK, CA",2018-02-10,https://d2g8igdw686xgo.cloudfront.net/27651948...,0.000000,296.0,0.9854,0
2,2,1543889806-34056,https://www.gofundme.com/mvc.php?route=homepag...,Home lost in California wildfire\n ...,https://www.gofundme.com/home-lost-in-californ...,Home lost in California wildfire,David Porter and his family lost their home an...,9350,25000.0,62,61,172,"PARADISE, CA",2018-11-10,https://d2g8igdw686xgo.cloudfront.net/34405786...,37.400000,23.0,0.6124,0
7,7,1543888990-33721,https://www.gofundme.com/mvc.php?route=homepag...,Blank Family Santa Rosa Fire Fund\n ...,https://www.gofundme.com/blank-santa-rosa-fire...,Blank Family Santa Rosa Fire Fund,"Our parents, Gary Blank and Carole Cooper, jus...",12780,30000.0,124,127,527,"SANTA ROSA, CA",2017-10-11,https://d2g8igdw686xgo.cloudfront.net/24575816...,42.600000,418.0,0.8068,0
8,8,1543889006-33728,https://www.gofundme.com/mvc.php?route=homepag...,A new home for Lindsay and Riley\n ...,https://www.gofundme.com/lindsayandriley,A new home for Lindsay and Riley,We are reaching out to help our dear friend Li...,29332,20000.0,130,138,661,"SANTA ROSA, CA",2017-10-19,https://d2g8igdw686xgo.cloudfront.net/24765678...,146.660000,410.0,0.9894,1
13,13,1543889112-33773,https://www.gofundme.com/mvc.php?route=homepag...,RainforestFlora Nursery & Home Fire\n ...,https://www.gofundme.com/lilacnursery,RainforestFlora Nursery & Home Fire,"My name is Kimberly Isley, I am from Southern ...",32385,35000.0,182,188,481,"BONSALL, CA",2017-12-08,https://d2g8igdw686xgo.cloudfront.net/26030326...,92.528571,360.0,0.3322,1
14,14,1543889125-33779,https://www.gofundme.com/mvc.php?route=homepag...,California Woolsey Wildfire Relief\n ...,https://www.gofundme.com/woolsey-wildfire-reli...,California Woolsey Wildfire Relief,The Woolsey Fire started on November 8th and h...,267,50000.0,12,13,23,"NORTHRIDGE, CA",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34528306...,0.534000,19.0,-0.6093,0
18,18,1543888468-33513,https://www.gofundme.com/mvc.php?route=homepag...,Assistance for fire victims\n ...,https://www.gofundme.com/wx5est-assistance-for...,Assistance for fire victims,California wildfire financial assistance for v...,0,15000.0,0,0,0,"RAPID CITY, SD",2018-11-13,https://d2g8igdw686xgo.cloudfront.net/34479628...,0.000000,20.0,-0.4404,0
22,22,1543889266-33834,https://www.gofundme.com/mvc.php?route=homepag...,John's California Wildfire Help Fun\n ...,https://www.gofundme.com/johns-california-wild...,John's California Wildfire Help Fund,I would like to get funds together to go help ...,0,250000.0,0,0,0,"TOLLESON, AZ",2017-12-05,https://d2g8igdw686xgo.cloudfront.net/25937554...,0.000000,363.0,0.8553,0
23,23,1543888130-33362,https://www.gofundme.com/mvc.php?route=homepag...,Shapers for Wildfire Relief Fund\n ...,https://www.gofundme.com/shapers-for-wildfire-...,Shapers for Wildfire Relief Fund,Join us at our fundraising events:\n- 11/27 Lo...,7625,15000.0,60,61,213,"SAN FRANCISCO, CA",2018-11-14,https://d2g8igdw686xgo.cloudfront.net/34540332...,50.833333,19.0,-0.7089,0


**LDA**

In [41]:
import pandas as pd
cali_posts = pd.read_csv('california_data.csv')
cali_posts=cali_posts.fillna('')

In [45]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

import nltk
nltk.download('wordnet')

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/roshnidas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
import re
import matplotlib.pyplot as plt
from google.oauth2 import service_account
from google.cloud import vision
import csv
import urllib.request
import time

cali_posts['image_labels']=''
def get_posts_attrs(cali_posts):
    credentials = service_account.Credentials.from_service_account_file('UGCA-f6b4b2de52f8.json')
    client = vision.ImageAnnotatorClient(credentials=credentials)
    image = vision.types.Image()
    for index,row in cali_posts.iterrows():
        if cali_posts['img'][index] == '':
            continue
        #print(cali_posts['img'][index])
        image.source.image_uri = cali_posts['img'][index]
        response = client.label_detection(image=image)
        cali_posts['image_labels'][index] = [label.description for label in response.label_annotations]
        

get_posts_attrs(cali_posts)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [61]:
cali_posts['image_labels']

0      [yellow, font, line, clock, circle, area, grap...
1      [property, sky, grass, real estate, land lot, ...
2      [fire, geological phenomenon, formation, bonfi...
3      [fire, sky, wildfire, heat, geological phenome...
4      [car, mode of transport, vehicle, road, recrea...
5      [geological phenomenon, heat, fire, flame, atm...
6                                                       
7      [geological phenomenon, pollution, disaster, t...
8      [horse, pasture, horse like mammal, mare, foal...
9      [cat, small to medium sized cats, whiskers, ca...
10     [product, event, glasses, fun, smile, recreati...
11     [wildfire, fire, explosion, sky, night, disast...
12     [hair, facial hair, photography, selfie, smile...
13       [tree, wood, village, scrap, landscape, rubble]
14     [fire, wildfire, firefighter, geological pheno...
15     [sky, geological phenomenon, atmosphere, red s...
16     [text, human behavior, history, advertising, b...
17         [fog, morning, sky, 

In [46]:
stopwords_nltk=set(stopwords.words('english'))

In [66]:
words_freq_vec = CountVectorizer(input=post_labels,stop_words=stopwords_nltk,decode_error='ignore')

In [67]:
labels_under_pro = words_freq_vec.fit_transform([' '.join(p) for p in post_labels])

In [69]:
ntopics = 4
# corpus_arr = np.array(corpus)
model = lda(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(labels_under_pro)

TypeError: __init__() got an unexpected keyword argument 'n_topics'

In [ ]:
type(labels_under_pro)
# dataframe_label = pd.DataFrame(total_features_words)
dataframe_label = pd.DataFrame({'post_no': range(len(post_labels)), 'label': post_labels})

In [ ]:
topic_label = model.topic_word_
topic_post=model.doc_topic_
topic_post=pd.DataFrame(topic_post)
dataframe_label=dataframe_label.join(topic_post)

In [ ]:
Image_Posts=pd.DataFrame()

In [ ]:
for i in range(int(ntopics)):
    topic="topic_"+str(i)
    Image_Posts[topic]=dataframe_label.groupby(['post_no'])[i].mean()

In [ ]:
Image_Posts=Image_Posts.reset_index()
topics=pd.DataFrame(topic_label)
topics.columns=words_freq_vec.get_feature_names()
topics1=topics.transpose()
print ("Topics label distribution written in file topic_label_dist.csv ")
topics1.to_csv("topic_label_dist.csv")
Image_Posts.to_csv("Image_posts_topic_dist.csv",index=False)
print ("Image topic distribution written in file Image_posts_topic_dist.csv ")

In [64]:
post_labels = [cali_posts['image_labels'][i] for i in range(len(cali_posts))]

In [65]:
post_labels

[['yellow',
  'font',
  'line',
  'clock',
  'circle',
  'area',
  'graphics',
  'product',
  'angle',
  'symbol'],
 ['property',
  'sky',
  'grass',
  'real estate',
  'land lot',
  'rural area',
  'landscape',
  'estate',
  'cloud',
  'tree'],
 ['fire',
  'geological phenomenon',
  'formation',
  'bonfire',
  'flame',
  'heat',
  'wildfire',
  'explosive material',
  'night',
  'cave'],
 ['fire',
  'sky',
  'wildfire',
  'heat',
  'geological phenomenon',
  'explosion',
  'atmosphere',
  'smoke',
  'evening',
  'explosive material'],
 ['car', 'mode of transport', 'vehicle', 'road', 'recreation', 'product'],
 ['geological phenomenon',
  'heat',
  'fire',
  'flame',
  'atmosphere',
  'explosion',
  'explosive material',
  'sky',
  'wildfire'],
 '',
 ['geological phenomenon',
  'pollution',
  'disaster',
  'tree',
  'soil',
  'waste',
  'rubble',
  'demolition',
  'village'],
 ['horse',
  'pasture',
  'horse like mammal',
  'mare',
  'foal',
  'grassland',
  'grazing',
  'grass',
  'pac